In [1]:
import numpy as np
import pandas as pd
from plotly import graph_objs as go

In [30]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

df = pd.read_csv('/content/Womens Clothing E-Commerce Reviews.csv')

In [31]:
df.head(10)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
7,7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits
8,8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses


In [32]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [33]:
del df['Unnamed: 0']
print(f'There are {df.shape[0]} reviews in the dataset,')
print(f'NUmber of duplicates : {len(df[df.duplicated()])}')
print(f'Number of missing values : {df.isnull().sum().sum()}')

There are 23486 reviews in the dataset,
NUmber of duplicates : 21
Number of missing values : 4697


In [34]:
print(f'Number of missing values per column : {df.isnull().sum().sort_values(ascending=True)}')

Number of missing values per column : Clothing ID                   0
Age                           0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
Review Text                 845
Title                      3810
dtype: int64


In [35]:
df = df.dropna(subset = ['Review Text','Division Name','Department Name','Class Name'])

In [36]:
recommended = (
    df['Recommended IND']
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={'index':'Recommended', 'Recommended IND':'Count'})
    .sort_values(by=['Recommended'], ascending=True)
    .replace([0, 1], ['No', 'Yes'])
          )

colors = ['#f6b220','#0E2F44']

fig = go.Figure(data=[go.Pie(labels=recommended['Recommended'],
                             values=recommended['Count'])])

fig.update_traces(hoverinfo='percent',
                  textinfo='label',
                  textfont_size=20,
                  marker=dict(colors=colors,
                              line=dict(color='white', width=1)))

fig.update_layout(showlegend=False,
                  title_text="<b>Recommended</b> Distribution",
                  title_x=0.5,
                  font=dict(family="Rockwell, sans-serif", size=25, color='#000000'))

fig.show()

In [37]:
classes = (
    df
    .groupby(['Recommended IND', 'Class Name'])
    .size()
    .to_frame()
    .rename(columns={0:'Count'})
    .reset_index()
          )

# get proportions in each class
a = classes.groupby('Class Name')['Count'].transform('sum')

classes['Count'] = classes['Count'].div(a)

# pivot table
classes = classes.pivot(index='Class Name', columns='Recommended IND')

fig = go.Figure()
fig.add_trace(go.Bar(
    y=classes.index,
    x=classes.iloc[:,0],
    name='Not Recommended',
    orientation='h',
    marker=dict(
        color='#f6b220')
    ))

fig.add_trace(go.Bar(
    y=classes.index,
    x=classes.iloc[:,1],
    name='Recommended',
    orientation='h',
    marker=dict(
        color='#0E2F44')
    ))

fig.update_layout(barmode='stack')

fig.update_layout(
                title = 'Distribution of <b>Product Class</b> by Recommendation ',
                barmode='stack',
                autosize=False,
                width=680,
                height=800,
                font=dict(family="Rockwell, sans-serif", size=18, color='#000000'),
                margin=dict(
                  l=150,
                  r=100,
                   b=30,
                   t=100,
                   pad=4
                          ))
fig.layout.xaxis.tickformat = ',.0%'

fig.show()

In [38]:
rating = (
    df['Rating']
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={'index':'Rating', 'Rating':'Count'})
    .sort_values(by=['Rating'], ascending=True)
          )

rating['percent'] = ((rating['Count'] / rating['Count'].sum())*100).round(2).astype(str) + '%'

colors = ['#0E2F44',] * 5
colors[4] = '#f6b220'

fig = go.Figure(go.Bar(
            y=rating['Count'],
            x=rating['Rating'],
            marker_color=colors,
            text=rating['percent']
                        ))

fig.update_traces(texttemplate='%{text}',
                  textposition='outside',
                  cliponaxis = False,
                  hovertemplate='<b>Rating</b>: %{x}<br><extra></extra>'+  # <extra></extra> removes trance 0
                                '<b>Count</b>: %{y}',
                  textfont_size=18)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.update_layout(coloraxis=dict(colorscale='Teal'),
                  showlegend=False,
                  plot_bgcolor='#F7F7F7',
                  margin=dict(pad=20),
                  paper_bgcolor='#F7F7F7',
                  height=500,
                  yaxis={'showticklabels': False},
                  yaxis_title=None,
                  xaxis_title=None,
                  title_text="<b>Rating</b> Distribution",
                  title_x=0.5,
                  font=dict(family="Rockwell, sans-serif", size=22, color='#000000'),
                  title_font_size=35)


fig.show()

# NLP

For the following models, I'll just be focusing on the review text and recommendation columns to determine if, based on the text, the user will recommend the product to other customers (i.e. if they like it enough to recommend it to some else). This way it will reduce the complexity of the model and make it a classification binary model.

In [39]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

import re
import string

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
# Let's just work with the reviews and recommendations
data = df[['Review Text', 'Recommended IND']]

In [41]:
# Cleaning the text
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,
    remove links, remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [42]:
data['Review Text'] = data['Review Text'].apply(lambda x:clean_text(x))

In [43]:
data

,Review Text,Recommended IND
0,absolutely wonderful silky and sexy and comfo...,1
1,love this dress its sooo pretty i happened t...,1
2,i had such high hopes for this dress and reall...,0
3,i love love love this jumpsuit its fun flirty ...,1
4,this shirt is very flattering to all due to th...,1
...,...,...
23481,i was very happy to snag this dress at such a ...,1
23482,it reminds me of maternity clothes soft stretc...,1
23483,this fit well but the top was very see through...,0
23484,i bought this dress for a wedding i have this ...,1


In [44]:
# Setting up the evaluation metrics
def roc_auc(predictions,target):

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [45]:
# Split target & features
X = data.drop('Recommended IND', axis=1)
y = data['Recommended IND']

# Spliting train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2,
                                                    random_state=42, shuffle=True)

# Tokenization

Tokenization is an essencial step on a NLP process. Tokenization is essentially splitting a phrase, sentence, paragraph, or an entire text document into smaller units, such as individual words or terms. Each of these smaller units are called tokens.


In [46]:
# Keras takenization text data prep

num_words = None   # the most X frequent words is returned

# Tokenize data
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train['Review Text'].tolist() + X_test['Review Text'].tolist())   # introduce text in list

# Get data word index
word_index = tokenizer.word_index

# Encode training/test data sentences into sequences
X_train_seq = tokenizer.texts_to_sequences(X_train['Review Text'].tolist())
X_test_seq = tokenizer.texts_to_sequences(X_test['Review Text'].tolist())

# Get max training sequence length
max_len = max([len(x) for x in X_train_seq])

# Pad the training/test sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Output some results
print("\nPadded training shape:", X_train_pad.shape)
print("\nPadded test shape:", X_test_pad.shape)


Padded training shape: (18102, 112)

Padded test shape: (4526, 112)


# RNN


In [47]:
# Basic RNN
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                    50,     # embeds it in a 50-dimensional vector
                    input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 112, 50)           1022500   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               15100     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1037701 (3.96 MB)
Trainable params: 1037701 (3.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
batch_size = 512

model.fit(X_train_pad, y_train, epochs=15, batch_size=batch_size)

Epoch 1/15
36/36 [==============================] - 19s 407ms/step - loss: 0.4825 - accuracy: 0.8113
Epoch 2/15
36/36 [==============================] - 9s 242ms/step - loss: 0.4482 - accuracy: 0.8189
Epoch 3/15
36/36 [==============================] - 10s 275ms/step - loss: 0.4963 - accuracy: 0.7675
Epoch 4/15
36/36 [==============================] - 11s 300ms/step - loss: 0.4175 - accuracy: 0.8294
Epoch 5/15
36/36 [==============================] - 10s 281ms/step - loss: 0.3567 - accuracy: 0.8438
Epoch 6/15
36/36 [==============================] - 8s 233ms/step - loss: 0.3305 - accuracy: 0.8571
Epoch 7/15
36/36 [==============================] - 10s 271ms/step - loss: 0.3063 - accuracy: 0.8724
Epoch 8/15
36/36 [==============================] - 10s 275ms/step - loss: 0.2802 - accuracy: 0.8831
Epoch 9/15
36/36 [==============================] - 10s 280ms/step - loss: 0.2427 - accuracy: 0.9005
Epoch 10/15
36/36 [==============================] - 9s 237ms/step - loss: 0.1893 - accuracy:

In [49]:
scores = model.predict(X_test_pad)
print("AUC: %.2f%%" % (roc_auc(scores,y_test)))

142/142 [==============================] - 3s 17ms/step
AUC: 0.89%


# LSTM's

In [50]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                    50,     # embeds it in a 50-dimensional vector
                    input_length=max_len))

model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 112, 50)           1022500   
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1083001 (4.13 MB)
Trainable params: 1083001 (4.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
batch_size = 512

model.fit(X_train_pad, y_train, epochs=5, batch_size=batch_size)

Epoch 1/5
36/36 [==============================] - 55s 1s/step - loss: 0.5125 - accuracy: 0.8129
Epoch 2/5
36/36 [==============================] - 52s 1s/step - loss: 0.3605 - accuracy: 0.8365
Epoch 3/5
36/36 [==============================] - 53s 1s/step - loss: 0.2905 - accuracy: 0.8763
Epoch 4/5
36/36 [==============================] - 51s 1s/step - loss: 0.2163 - accuracy: 0.9084
Epoch 5/5
36/36 [==============================] - 50s 1s/step - loss: 0.1842 - accuracy: 0.9234


In [52]:
scores = model.predict(X_test_pad)
print("AUC: %.2f%%" % (roc_auc(scores,y_test)))

142/142 [==============================] - 7s 45ms/step
AUC: 0.93%
